#### Configuração do ambiente
O passo inicial é preparar o ambiente do Google Colab para que ele tenha acesso a todos os módulos e arquivos do nosso projeto. Para garantir a praticidade e o uso da versão mais atualizada do código, optamos por clonar o repositório diretamente do GitHub.


* **1. Clonagem do repositório:** utilizamos o comando `!git clone`. Este comando baixa uma cópia completa e privada do nosso projeto (`zerionkit`) para o ambiente de execução temporário do Colab. O acesso é feito de forma segura utilizando um *Personal Access Token (PAT)* do GitHub.

In [ ]:
!git clone https://github.com/petribrn/zerionkit

Cloning into 'zerionkit'...
remote: Enumerating objects: 255, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 255 (delta 0), reused 0 (delta 0), pack-reused 228 (from 1)
Receiving objects: 100% (255/255), 1.33 MiB | 16.23 MiB/s, done.
Resolving deltas: 100% (126/126), done.


* **2. Mudança de diretório:** este comando muda o diretório de trabalho atual para dentro da pasta do projeto clonado. Isso garante que quaisquer caminhos de arquivo relativos funcionem corretamente.


In [ ]:
%cd /content/zerionkit/

/content/zerionkit


* **3. Adição ao `sys.path` do python:** informa ao interpretador python onde encontrar nossos módulos customizados.


In [ ]:
import sys

project_path = '/content/zerionkit/'

sys.path.append(project_path)

* **4. Importações:** importamos toda as bibliotecas e classes necessárias para a execução do notebook.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from src.zerion_nn import ZerionNN
from src.layer import Layer
from src.data_handler import DataHandler
from src.loss import Loss

print("Módulos importados com sucesso!")

Módulos importados com sucesso!


####Carregamento e Preparação dos Dados
Utilizamos nossa classe `DataHandler` para carregar o dataset, dividi-lo em conjuntos de treino e validação, e realizar o pré-processamento necessário. O conjunto de dados de teste será carregado apenas na etapa final.

In [ ]:
training_data_handler = DataHandler(
        dataset_type='training',
        problem_type='regression',
        dataset_name='bike',
        y_target_columns='cnt',
    )
training_inputs, training_y_targets = training_data_handler.processed_data

validation_data_handler = DataHandler(
        dataset_type='validation',
        problem_type='regression',
        dataset_name='bike',
        y_target_columns='cnt',
    )
validation_inputs, validation_y_targets = validation_data_handler.processed_data

print("Dados carregados com sucesso!")

Dados carregados com sucesso!


####Definição e instanciação do modelo

Definimos a arquitetura da nossa rede neural (número de camadas, neurônios e funções de ativação) e outros hiperparâmetros como a taxa de aprendizado e o número de épocas.

In [ ]:
neural_network = ZerionNN(
        problem_type='regression',
        layers=[
            Layer(size=len(training_inputs[0]), activation='linear'),
            Layer(size=8, activation='relu'),
            Layer(size=len(training_y_targets.keys()), activation='linear'),
        ],
        loss='square_error',
        learning_rate=0.001,
        epochs=10,
    )

print("Rede neural criada com sucesso")

Rede neural criada com sucesso


#### Treinamento do modelo
Treinamos o modelo apenas com o conjunto de dados de treino, utilizando o método `.train()`.



In [ ]:
metrics = neural_network.train(
        inputs=training_inputs,
        y_targets=training_y_targets,
    )

#### Desempenho do modelo
Analisamos o desempenho do modelo que acabamos de treinar. Primeiro, visualizamos a curva de aprendizado para garantir que o treinamento ocorreu de forma estável. Em seguida, avaliamos a performance do modelo nos conjuntos de validação e teste para medir sua capacidade de generalização em dados novos.


* **1. Gráfico da curva de aprendizado:** plotamos o erro e a acurácia do treinamento ao longo das épocas, mostrando como o modelo aprendeu.

In [ ]:
errors = metrics['errors']
accuracies = metrics['accuracies']
epochs_range = range(len(errors))

# Criar uma figura com dois subplots
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10), sharex=True)

# Plotar Erro de Treinamento
ax1.plot(epochs_range, errors, 'r-', label='Training Error (Loss)')
ax1.set_ylabel('Error (Loss)')
ax1.set_title('Training Error and Accuracy Over Epochs')
ax1.grid(True)
ax1.legend()

# Plotar Acurácia de Treinamento
ax2.plot(epochs_range, accuracies, 'b-', label='Training Accuracy')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy (%)')
ax2.grid(True)
ax2.legend()

plt.show()

* **2. Avaliação nos dados de validação:** usamos o método `.evaluate()` para obter as métricas finais no conjunto de validação.


In [ ]:
neural_network.evaluate(
        inputs=validation_inputs,
        y_targets=validation_y_targets,
        y_scaler=validation_data_handler.y_scaler,
    )

* **3. Avaliação nos dados de teste:** carregamos e preparamos o conjunto de dados de teste. Também, usamos o método `.evaluate()` para obter as métricas finais no conjunto de teste.

In [ ]:
test_data_handler = DataHandler(
        dataset_type='test',
        problem_type='regression',
        dataset_name='bike',
        y_target_columns='cnt',
    )
test_inputs, test_y_targets = test_data_handler.processed_data

neural_network.evaluate(
    inputs=test_inputs,
    y_targets=test_y_targets,
    y_scaler=test_data_handler.y_scaler,
)